In [1]:
import requests
from langfuse import Langfuse

import os
import sys
sys.path.append(os.path.abspath("C:/вузик/sii/bachelor-2025-team-losoci/ml"))
import prompt_templates


### Параметры

In [2]:
MODEL_CONFIG = {
    "model_name": "llama3.1",
    "base_url": "http://26.81.6.105:11434/api/generate",
    "langfuse_secret_key": "sk-lf-03128ca3-f1c4-4e1b-ba7c-17b8cf50f892",
    "langfuse_public_key": "pk-lf-76c5e16f-7acd-420f-9e29-6d0f59507b4a", 
    "langfuse_host": "http://26.81.6.105:3000"
}

In [3]:
langfuse = Langfuse(
    secret_key=MODEL_CONFIG["langfuse_secret_key"],
    public_key=MODEL_CONFIG["langfuse_public_key"],
    host=MODEL_CONFIG["langfuse_host"]
)

### Функции

#### Курс доллара

In [4]:
import requests

def get_currency_rate():
    
    url = "https://api.exchangerate-api.com/v4/latest/USD"
    #try:
    response = requests.get(url)
    data = response.json()
    rub_rate = data['rates']['RUB']
    return rub_rate
    #except Exception as e:
        #print(f"Ошибка при получении курса: {e}")
        #return None

#### Отправка запроса

In [5]:
def generate_response(prompt: str, 
                     system: str,
                     model_name: str = None,
                     max_tokens: int = 500,
                     temperature: float = 0.7 ):
                     
    model = model_name or MODEL_CONFIG["model_name"]
    system_content = system

    with langfuse.start_as_current_generation(
        name=f"LLM-Call-{model}",
        model=model,
        input={
            "system": system_content,
            "prompt": prompt},
        model_parameters={
            "temperature": temperature,
            "max_tokens": max_tokens
        }
    ) as generation:
        data = {
            "model": model,
            "prompt": prompt,
            "system": system_content,
            "stream": False,
            "options": {
                "temperature": temperature,
                "max_tokens": max_tokens
            }
        }
                    
        response = requests.post(MODEL_CONFIG["base_url"], json=data)
    
        if response.status_code == 200:
            full_response = response.json()
            generated_text = full_response["response"]

            # Подсчет токенов
            prompt_tokens = full_response.get("prompt_eval_count", 0)
            completion_tokens = full_response.get("eval_count", 0)

            # Подсчет стоимости токена из расчета
            # rate = get_currency_rate()
            currency_rate = 83
            input_cost_per_token = 0.00000244 * currency_rate
            output_cost_per_token = input_cost_per_token * 2.5 
            
            input_cost = prompt_tokens * input_cost_per_token
            output_cost = completion_tokens * output_cost_per_token
            total_cost = input_cost + output_cost

            generation.update(
                output=generated_text,
                usage_details={
                    "input": prompt_tokens,
                    "output": completion_tokens,
                },
                cost_details={
                    "input": input_cost,
                    "output": output_cost,
                    "total": total_cost 
                },
                metadata={
                    "response_time": full_response.get("total_duration", 0),
                    "model": model,
                    "system": system_content,
                    "prompt": prompt,
                    "stream": False,
                    "options": {
                        "temperature": temperature,
                        "max_tokens": max_tokens
                    }
                }
                
            )
            
            return generated_text
            
        else:
            error_msg = f"Ошибка API: {response.status_code} - {response.text}"
            
            generation.update(
                output={"error": error_msg},
                level="ERROR"
            )

            return {
                "success": False,
                "error": error_msg,
                "model": model
            }
            
    langfuse.flush()

### Запрос модели

In [6]:
generate_response(prompt_templates.prompts["balanced"], prompt_templates.system_content[1])

AttributeError: module 'prompt_templates' has no attribute 'prompts'